In [3]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, sefara, varcode, uncertainties, numpy, statsmodels, pickle, scipy
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import matplotlib_venn
import intervaltree
import statsmodels.robust
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import scipy, sys, sklearn
import sklearn.utils
import matplotlib
#import common as c
import sklearn.tree
import pymc3
from scipy import optimize
import glob


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'

import pystan
import pyfaidx



In [4]:
# Write out mutation counts
mutations = pandas.read_csv("../data/mutations.csv.bz2", converters={'binding_peptides': eval})
mutations = mutations.ix[mutations.source_id.str.startswith("AOCS")]
mutations

,source_id,donor,called,genome,contig,interbase_start,interbase_end,ref,alt,effect,gene,context_5_prime,context_3_prime,context_mutation,dna_alt_reads,dna_ref_reads,dna_total_reads,rna_alt_reads,rna_ref_reads,rna_total_reads,vaf,any_alt_reads,snv,mutation_id,site_id,context_A,context_C,context_T,context_G,context_GC,unique_to_treated,context_mutation_5p,context_mutation_3p,context_mutation_3p5p,sources,binding_affinity,binding_allele,num_alt,num_ref,total_depth,binder,binding_peptides,num_binders
0,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,61407797,61407798,C,T,intergenic,None,CCCCTCATCTCCCAG,CTGAACCACGTCCTT,C>T,0,47,47,NaN,NaN,NaN,0.000000,False,True,18:61407797 C>T,18:61407797,0.166667,0.500000,0.233333,0.100000,0.600000,True,G(C>T),(C>T)C,G(C>T)C,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
1,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,18,61407797,61407798,C,T,intergenic,None,CCCCTCATCTCCCAG,CTGAACCACGTCCTT,C>T,5,19,24,NaN,NaN,NaN,0.208332,True,True,18:61407797 C>T,18:61407797,0.166667,0.500000,0.233333,0.100000,0.600000,True,G(C>T),(C>T)C,G(C>T)C,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
2,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,6,63262672,63262673,C,A,intergenic,None,GACCACACAGGAACC,GAGAATTTACATGCA,C>A,0,69,69,NaN,NaN,NaN,0.000000,False,True,6:63262672 C>A,6:63262672,0.400000,0.266667,0.133333,0.200000,0.466667,True,C(C>A),(C>A)G,C(C>A)G,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
3,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,6,63262672,63262673,C,A,intergenic,None,GACCACACAGGAACC,GAGAATTTACATGCA,C>A,9,40,49,NaN,NaN,NaN,0.183673,True,True,6:63262672 C>A,6:63262672,0.400000,0.266667,0.133333,0.200000,0.466667,True,C(C>A),(C>A)G,C(C>A)G,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
4,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,63487173,63487174,C,G,intronic,CDH7,TGTTCCTTTGCTTCC,TAAAGTGCGAAAGAT,C>G,0,31,31,NaN,NaN,NaN,0.000000,False,True,18:63487173 C>G,18:63487173,0.233333,0.200000,0.366667,0.200000,0.400000,True,C(C>G),(C>G)T,C(C>G)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
5,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,18,63487173,63487174,C,G,intronic,CDH7,TGTTCCTTTGCTTCC,TAAAGTGCGAAAGAT,C>G,6,17,23,NaN,NaN,NaN,0.260868,True,True,18:63487173 C>G,18:63487173,0.233333,0.200000,0.366667,0.200000,0.400000,True,C(C>G),(C>G)T,C(C>G)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
6,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,68110672,68110673,A,T,intergenic,None,TCCTTGTCGTCTCGT,TCTAGCCTGCAAACC,T>A,5,27,32,NaN,NaN,NaN,0.156250,True,True,18:68110672 A>T,18:68110672,0.133333,0.366667,0.333333,0.166667,0.533333,True,T(T>A),(T>A)T,T(T>A)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
7,AOCS-139-16/SP101896,AOCS-139,0,GRCh37,18,68110672,68110673,A,T,intergenic,None,TCCTTGTCGTCTCGT,TCTAGCCTGCAAACC,T>A,11,33,44,NaN,NaN,NaN,0.249999,True,True,18:68110672 A>T,18:68110672,0.133333,0.366667,0.333333,0.166667,0.533333,True,T(T>A),(T>A)T,T(T>A)T,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
8,AOCS-139-16/SP101896,AOCS-139,1,GRCh37,18,68385066,68385067,C,G,intergenic,None,TCCAGACAGACCACA,AGATTCACAGCCGAA,C>G,12,27,39,NaN,NaN,NaN,0.307692,True,True,18:68385066 C>G,18:68385066,0.400000,0.333333,0.100000,0.166667,0.500000,True,A(C>G),(C>G)A,A(C>G)A,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0
9,AOCS-139-4/SP101906,AOCS-139,1,GRCh37,18,68385066,68385067,C,G,intergenic,None,TCCAGACAGACCACA,AGATTCACAGCCGAA,C>G,9,43,52,NaN,NaN,NaN,0.173077,True,True,18:68385066 C>G,18:68385066,0.400000,0.333333,0.100000,0.166667,0.500000,True,A(C>G),(C>G)A,A(C>G)A,mutations_with_mnvs mutations_with_mnvs mutati...,NaN,NaN,NaN,NaN,NaN,False,{},0


In [5]:
sources = pandas.read_csv("../data/sources.csv", index_col="source_id")
sources["source_id"] = sources.index
sources = sources.ix[sources.cohort == "AOCS"]
treated_paired_donors = set(d for d in sources.donor
                              if len(sources.ix[sources.donor == d].treated.unique()) > 1 )
treated_paired_donors
sources["treated_paired"] = sources.donor.isin(treated_paired_donors)
sources

,donor,cohort,library,treated,timepoint,metastasis,tissue_type,interval_days,assay,paired,source_id,treated_paired
source_id,,,,,,,,,,,,
AOCS-001-1/SP101515,AOCS-001,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-001-1/SP101515,False
AOCS-002-1/SP101517,AOCS-002,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-002-1/SP101517,False
AOCS-004-1/SP101519,AOCS-004,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-004-1/SP101519,False
AOCS-005-1/SP101521,AOCS-005,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-005-1/SP101521,False
AOCS-034-13/SP101524,AOCS-034,AOCS,WGS,True,recurrence,False,ascites,1597,WGS,True,AOCS-034-13/SP101524,True
AOCS-034-1/SP101523,AOCS-034,AOCS,WGS,False,primary,False,solid,0,WGS,True,AOCS-034-1/SP101523,True
AOCS-055-1/SP101526,AOCS-055,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-055-1/SP101526,False
AOCS-056-1/SP101528,AOCS-056,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-056-1/SP101528,False
AOCS-057-1/SP101530,AOCS-057,AOCS,WGS,False,primary,False,solid,0,WGS,False,AOCS-057-1/SP101530,False


In [6]:
muts_new = mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().fillna(0).unstack().fillna(0)
muts_new["kind"] = "new"
muts_new

context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-034-13/SP101524,69,85,7,65,72,44,16,86,80,33,58,72,41,30,21,60,76,25,37,42,23,5,39,35,84,61,10,120,44,38,13,82,69,286,36,267,47,45,55,123,36,45,39,59,19,16,37,29,68,77,5,44,46,49,5,72,82,76,43,68,13,18,22,32,35,32,25,41,12,12,22,19,100,76,10,99,331,82,18,495,266,97,41,187,37,33,25,55,49,39,15,38,32,18,38,70,new
AOCS-064-13/SP101554,145,157,11,178,127,61,32,138,172,82,65,133,79,74,80,127,108,69,73,145,50,23,66,69,120,105,13,124,78,58,24,80,81,82,58,194,127,103,127,219,56,118,86,258,34,44,75,193,152,152,9,91,62,157,6,117,124,146,42,106,54,50,40,104,50,91,42,85,17,49,50,80,79,98,10,114,100,93,18,182,87,66,36,120,84,73,42,186,96,111,37,129,51,38,70,162,new
AOCS-065-13/SP101560,27,23,3,30,24,15,5,43,33,18,24,26,17,10,15,32,26,10,11,28,9,4,14,8,28,22,3,27,24,12,5,28,30,44,21,48,18,21,13,28,6,13,15,22,7,2,13,24,27,14,1,20,9,17,1,28,30,30,24,27,9,14,15,10,14,12,1,18,11,4,12,12,28,27,2,26,33,17,2,38,25,30,16,26,7,18,10,21,21,10,6,25,18,7,13,20,new
AOCS-086-13/SP101606,97,125,6,78,83,42,23,93,81,56,87,95,44,29,41,59,85,34,64,93,32,11,33,29,90,79,11,128,62,49,21,88,126,363,49,317,112,103,131,190,58,64,64,109,23,20,44,53,118,127,13,79,46,118,9,115,120,109,65,107,16,30,24,39,47,53,41,48,16,13,30,25,73,71,8,100,122,76,11,206,107,81,34,105,72,39,25,77,60,61,31,72,34,14,48,77,new
AOCS-088-13/SP101612,203,230,17,205,134,78,27,125,189,98,94,199,78,64,83,144,123,85,73,143,43,18,74,49,176,173,21,201,89,74,19,118,123,138,66,232,111,144,137,234,63,135,60,85,24,32,74,72,127,115,15,148,60,71,11,92,139,89,79,131,62,74,69,141,56,51,42,63,35,16,64,53,210,154,21,260,327,155,32,513,413,189,47,323,97,93,61,207,90,99,45,98,46,38,94,87,new
AOCS-091-13/SP101624,139,166,15,121,167,63,23,120,159,75,124,129,63,69,52,97,137,71,112,125,31,18,55,37,138,127,12,172,81,44,12,128,83,165,93,192,133,120,175,221,64,83,68,113,31,30,60,51,98,146,16,88,63,92,8,129,155,103,92,105,45,47,48,80,65,76,53,89,22,22,58,40,111,100,9,127,121,92,11,213,107,86,50,107,98,74,50,140,100,85,49,100,52,30,56,80,new
AOCS-092-13/SP101630,75,161,2,88,45,18,5,43,53,26,54,56,41,51,28,44,42,29,32,49,23,10,22,34,112,77,21,203,22,24,10,64,149,543,46,524,315,172,268,360,55,43,48,106,21,19,37,30,82,154,6,68,18,111,3,64,109,107,48,58,22,27,15,29,23,72,19,27,8,24,12,19,57,67,8,111,54,32,8,134,80,91,37,115,41,28,24,48,28,35,17,41,9,16,29,42,new
AOCS-093-13/SP101638,57,81,4,56,48,24,13,40,49,34,29,57,28,12,38,40,65,34,43,53,11,8,21,20,37,48,4,65,30,47,15,55,56,127,26,126,83,57,91,81,34,41,42,56,11,14,33,25,53,69,8,33,22,60,7,51,69,54,24,47,15,17,29,22,26,37,21,28,7,8,14,10,49,37,7,64,89,60,6,130,101,53,18,74,30,16,19,46,42,34,21,47,24,18,25,42,new
AOCS-094-2/SP106641,95,104,9,102,96,61,26,105,116,53,65,126,43,42,45,86,93,59,71,102,35,10,42,24,94,93,6,85,59,53,18,93,56,41,49,107,48,66,62,99,45,109,52,64,15,29,57,28,46,58,9,67,40,32,9,52,68,47,43,65,28,26,25,52,48,46,40,47,21,7,39,30,96,53,9,95,170,86,19,227,88,58,28,100,70,37,38,87,80,108,46,112,38,25,55,73,new


In [7]:
muts_all = mutations.ix[
    mutations.snv
].groupby(["source_id", "context_mutation_3p5p"]).interbase_start.count().unstack().T.fillna(0)
#muts_all.columns = ["%s__all" % (x.replace("/", "_")) for x in muts_all.columns]
muts_all = muts_all.T
muts_all["kind"] = "all"

print(muts_all.isnull().sum().sum())
muts_all


0


context_mutation_3p5p,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T,kind
source_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-001-1/SP101515,121,85,15,92,84,48,12,83,93,53,123,105,30,32,44,60,125,48,81,120,24,15,32,32,99,86,13,81,52,40,12,72,72,75,74,110,44,60,72,68,35,70,49,44,17,16,25,31,65,69,8,56,42,42,5,40,89,76,83,67,29,19,42,54,54,29,47,67,4,5,34,27,78,94,19,103,149,89,12,235,158,109,64,127,51,51,28,91,67,47,37,81,22,21,35,63,all
AOCS-002-1/SP101517,37,22,6,24,17,8,5,11,30,21,68,29,16,18,13,17,49,12,35,51,13,3,8,9,26,16,1,16,7,8,5,10,26,34,56,41,5,7,10,11,9,9,9,15,2,5,4,10,11,4,4,14,7,8,0,4,18,23,46,28,4,6,9,9,15,11,6,13,2,3,4,8,23,25,5,28,14,6,2,23,26,33,24,43,12,8,8,29,17,4,7,20,6,6,8,16,all
AOCS-004-1/SP101519,109,108,17,83,78,47,12,72,138,68,131,100,54,57,63,83,155,59,103,155,23,15,34,33,125,79,13,133,57,51,26,79,99,77,103,132,36,44,56,78,85,80,85,118,22,31,46,93,50,44,9,56,39,33,8,47,90,79,123,108,39,34,39,48,93,61,75,110,17,12,31,39,67,77,10,117,59,68,15,147,142,102,57,125,80,42,42,104,112,93,41,95,27,28,62,69,all
AOCS-005-1/SP101521,107,66,6,60,55,38,18,53,123,55,167,101,49,26,32,71,125,57,100,127,32,17,37,53,56,44,9,45,43,32,14,56,84,59,143,85,23,35,32,52,43,51,45,73,22,18,31,59,66,43,10,53,25,18,10,31,77,67,134,62,24,11,13,31,50,25,33,50,25,10,43,35,85,83,21,117,148,74,11,252,206,102,91,178,61,14,19,66,58,50,37,80,36,20,44,84,all
AOCS-034-1/SP101523,166,103,16,95,116,57,16,107,162,75,144,104,57,40,50,96,188,64,73,113,44,16,30,33,91,97,15,99,67,56,22,106,95,78,93,113,47,64,67,72,49,54,49,104,19,32,52,47,70,66,10,71,49,43,10,77,109,71,105,84,35,29,44,54,64,30,64,51,34,32,279,45,97,130,16,117,169,94,12,252,170,117,74,147,69,39,33,104,97,78,35,93,43,21,44,92,all
AOCS-034-13/SP101524,236,198,27,161,189,98,30,186,240,109,195,167,92,65,72,146,263,83,105,151,67,21,65,67,178,162,25,231,110,95,32,192,171,417,133,421,95,106,123,202,90,99,92,161,38,48,89,71,141,172,16,115,95,103,16,159,188,153,157,158,49,47,69,89,89,63,87,86,43,43,301,63,209,215,26,222,513,182,28,785,472,213,124,335,106,68,57,156,137,109,47,130,74,38,79,162,all
AOCS-055-1/SP101526,152,122,17,137,147,78,42,149,184,94,318,144,55,51,63,115,247,109,168,285,43,8,57,64,130,91,14,120,124,90,38,147,126,113,183,203,50,53,76,101,103,140,96,155,39,48,82,121,89,76,19,70,65,59,15,109,140,97,222,107,41,39,43,76,117,73,89,140,34,24,71,62,263,152,27,197,1617,385,97,1656,1042,344,227,607,93,50,51,129,136,121,71,146,87,39,66,152,all
AOCS-056-1/SP101528,181,160,15,130,130,58,16,108,144,77,187,137,70,60,97,90,126,63,96,117,21,13,53,35,109,123,13,101,77,49,18,85,72,86,122,149,47,81,77,104,58,72,35,57,22,30,58,46,88,96,16,78,37,33,11,74,107,87,125,95,36,39,43,79,51,26,42,60,21,7,51,29,97,126,9,102,78,74,8,167,107,85,90,114,67,58,38,124,60,59,39,86,25,17,48,79,all
AOCS-057-1/SP101530,120,121,6,98,105,34,10,96,132,66,134,93,59,31,56,76,136,53,84,116,32,10,29,28,91,109,17,105,49,37,9,80,79,66,99,97,42,77,57,73,55,75,51,88,14,22,46,41,63,46,19,57,30,34,7,55,93,77,100,74,31,31,41,64,52,41,51,53,17,10,36,25,93,84,12,120,86,72,14,155,92,106,66,87,71,39,45,107,81,69,38,91,28,16,51,66,all


In [10]:
# STATEMENT_UNIQUE_TO_TREATED_COUNT
print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors) &
    mutations.unique_to_treated
].shape)

print(mutations.ix[[
    (sources.ix[s].treated) for s in mutations.source_id
]].ix[
    mutations.snv &
    mutations.donor.isin(treated_paired_donors)
].shape)

(102377, 43)
(222709, 43)


In [ ]:
# To be loaded into R to use deconstructSigs package
muts_r = pandas.concat([muts_all, muts_new])

muts_r.columns = [x.replace("(", "[").replace(")", "]") for x in muts_r.columns]
muts_r["source_id"] = muts_r.index
muts_r = muts_r[["source_id"] + list(muts_r.columns)[:-1]]
muts_r.to_csv("../data/mutation_contexts_counts.csv", index=False)
muts_r